In [3]:
import csv
import random

input_file = 'Y_train.csv'
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

title = data[0]

rows = data[1:]

for i in range(24):
    extracted_rows = [rows[j] for j in range(len(rows)) if (j - i) % 24 == 0]
    
    output_file = f'./generated_data/output_{i+1}.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title)
        writer.writerows(extracted_rows)

    with open(output_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    
    title_row = data[0]
    rows_data = data[1:]
    
    for row in rows_data:
        original_value = float(row[0])
        for j in range(1, 2):
            random_factor = random.uniform(-0.15, 0.15)
            new_value = original_value * (1 + random_factor)
            row.append(f"{new_value:.2f}")
    
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title_row + ['Change1'])
        writer.writerows(rows_data)


In [4]:
import csv
import numpy as np
import os

mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]

ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1] / sum(mpc_bus[j][1] for j in range(118))

parent_folder = './hour'
if not os.path.exists(parent_folder):
    os.makedirs(parent_folder)

for file_num in range(1, 25):
    input_file = f'./generated_data/output_{file_num}.csv'
    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    title = data[0]
    rows = data[1:]

    folder_path = f'{parent_folder}/{file_num}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    allocated_data = []
    for row in rows:
        total_value = float(row[0])  
        allocated_row = [total_value * ratio_bus[i] for i in range(118)]
        allocated_data.append(allocated_row)

    output_file = f'{folder_path}/allocated_output_perfect.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Node_' + str(i+1) for i in range(118)])  
        writer.writerows(allocated_data)

    for col in range(1, 2):
        allocated_data = []
        for row in rows:
            total_value = float(row[col])  
            allocated_row = [total_value * ratio_bus[i] for i in range(118)]
            allocated_data.append(allocated_row)

        output_file = f'{folder_path}/allocated_output_change{col}.csv'
        with open(output_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Node_' + str(i+1) for i in range(118)]) 
            writer.writerows(allocated_data)



In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
def calculate_optimization_cost_two_stage(P_l_predict, Y_real, topo, unit, reserve_up=150, reserve_down=150, c=9000, c_s=1750):

    P_l_predict = np.array(P_l_predict).T    
    Y_real = np.array(Y_real).T
    
    nodes = topo.shape[0]
    
    model_DA = gp.Model()
    
    P_c = model_DA.addVars(nodes,  lb=0, name='P_c')       
    R_U = model_DA.addVars(nodes,  lb=0, name='R_U')       
    R_D = model_DA.addVars(nodes,  lb=0, name='R_D')       
    delta_DA = model_DA.addVars(nodes,  lb=-180, ub=180, name='delta_DA') 
    
    obj_DA = gp.quicksum(
        R_U[i]*unit.iloc[i,5] + 
        R_D[i]*unit.iloc[i,6] + 
        P_c[i]*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    model_DA.setObjective(obj_DA, GRB.MINIMIZE)
    

    model_DA.addConstr(gp.quicksum(R_U[i] for i in range(nodes)) == reserve_up)
    model_DA.addConstr(gp.quicksum(R_D[i] for i in range(nodes)) == reserve_down)
    for i in range(nodes):
        model_DA.addConstr(R_U[i] <= unit.iloc[i,3])  
        model_DA.addConstr(R_D[i] <= unit.iloc[i,4])  

    for t in range(nodes):
        model_DA.addConstr(
            P_c[t] - P_l_predict[t] - 
            gp.quicksum(topo.iloc[t,j]*(delta_DA[t]-delta_DA[j]) for j in range(nodes)) == 0
        )
    model_DA.addConstr(delta_DA[0] == 0)  
       
    for i in range(nodes):
        model_DA.addConstr(P_c[i] <= unit.iloc[i,1] - R_U[i])  
        model_DA.addConstr(P_c[i] >= R_D[i])                   
        
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) <= 175)
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) >= -175)
    
    model_DA.optimize()
    
    model_RT = gp.Model()
    
    
    r_U = model_RT.addVars(nodes,  lb=0, name='r_U')       
    r_D = model_RT.addVars(nodes,  lb=0, name='r_D')       
    P_lsh = model_RT.addVars(nodes,  lb=0, name='P_lsh')  
    delta_RT = model_RT.addVars(nodes,  lb=-180, ub=180, name='delta_RT') 
    s = model_RT.addVars(nodes,  lb=0, name='s')           
    
    
    obj_RT = gp.quicksum(
        (r_U[i]-r_D[i])*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    obj_RT += gp.quicksum(P_lsh[i]*c for i in range(nodes) )
    obj_RT += gp.quicksum(s[i]*c_s for i in range(nodes) )
    model_RT.setObjective(obj_RT, GRB.MINIMIZE)
    
    
    for t in range(nodes):
        model_RT.addConstr(
            r_U[t] - r_D[t] - s[t] - Y_real[t] + P_l_predict[t] + P_lsh[t] - 
            gp.quicksum(
                topo.iloc[t,j]*(delta_RT[t]-delta_RT[j]) - 
                topo.iloc[t,j]*(delta_DA[t].X-delta_DA[j].X) 
                for j in range(nodes)
            ) == 0
        )
        
    for i in range(nodes):
        model_RT.addConstr(r_U[i] <= R_U[i].X) 
        model_RT.addConstr(r_D[i] <= R_D[i].X)  
    model_RT.addConstr(delta_RT[0] == 0)  
        
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) <= 175)
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) >= -175)
    
    model_RT.optimize()
    
    
    total_cost = model_DA.ObjVal + model_RT.ObjVal
    
    
    
    model_DA.dispose()
    model_RT.dispose()
    
    return total_cost
# System parameters (topology data and generator unit data)
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')

hour_folder = './hour'
for hour in range(1, 25):
    hour_dir = os.path.join(hour_folder, str(hour))
    if not os.path.exists(hour_dir):
        continue
    

    perfect_file = os.path.join(hour_dir, 'allocated_output_perfect.csv')
    if not os.path.exists(perfect_file):
        continue
    

    with open(perfect_file, 'r') as f_perfect:
        reader_perfect = csv.reader(f_perfect)
        next(reader_perfect)  
        perfect_data = [list(map(float, row)) for row in reader_perfect]
    
    
    perfect_costs = []
    with ThreadPoolExecutor() as executor:
        
        perfect_costs = list(executor.map(lambda x: calculate_optimization_cost_two_stage(x, x, topo, unit), perfect_data))
    
    
    perfect_cost_file = os.path.join(hour_dir, 'perfect_costs.csv')
    with open(perfect_cost_file, 'w', newline='') as f_perfect_cost:
        writer = csv.writer(f_perfect_cost)
        writer.writerow(['Perfect_Cost'])
        writer.writerows([[cost] for cost in perfect_costs])
    
    
    cost_differences = []  
    for file_prefix in ['allocated_output_change1']:
        input_file = os.path.join(hour_dir, f'{file_prefix}.csv')
        if not os.path.exists(input_file):
            continue
        
        
        with open(input_file, 'r') as f_in:
            reader = csv.reader(f_in)
            next(reader)  
            data = [list(map(float, row)) for row in reader]
        
        
        total_costs = []
        with ThreadPoolExecutor() as executor:
            total_costs = list(executor.map(lambda x: calculate_optimization_cost_two_stage(x[0], x[1], topo, unit), zip(data, perfect_data)))
        
        cost_diff = [total_costs[i] - perfect_costs[i] for i in range(len(total_costs))]
        cost_differences.append(cost_diff)
    
    if cost_differences:
        output_file = os.path.join(hour_dir, 'cost_differences.csv')
        with open(output_file, 'w', newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(['Change1'])
            writer.writerows(zip(*cost_differences))

In [1]:
import os
import pandas as pd

input_folder = './generated_data'  
output_folder = 'load_variation'  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(1, 25):
    input_file = os.path.join(input_folder, f'output_{i}.csv')
    output_file = os.path.join(output_folder, f'load_variation_{i}.csv')
    
    if os.path.exists(input_file):
        df = pd.read_csv(input_file)
        
        
        required_columns = ['load (MW)', 'Change1']
        if all(col in df.columns for col in required_columns):
            
            load_variation = df[['Change1']].sub(df['load (MW)'], axis=0)
            
            
            load_variation.to_csv(output_file, index=False)
            print(f" {input_file} complete, save to {output_file}")
        else:
            print(f" {input_file} skip")
    else:
        print(f" {input_file} not exist")

 ./generated_data\output_1.csv complete, save to load_variation\load_variation_1.csv
 ./generated_data\output_2.csv complete, save to load_variation\load_variation_2.csv
 ./generated_data\output_3.csv complete, save to load_variation\load_variation_3.csv
 ./generated_data\output_4.csv complete, save to load_variation\load_variation_4.csv
 ./generated_data\output_5.csv complete, save to load_variation\load_variation_5.csv
 ./generated_data\output_6.csv complete, save to load_variation\load_variation_6.csv
 ./generated_data\output_7.csv complete, save to load_variation\load_variation_7.csv
 ./generated_data\output_8.csv complete, save to load_variation\load_variation_8.csv
 ./generated_data\output_9.csv complete, save to load_variation\load_variation_9.csv
 ./generated_data\output_10.csv complete, save to load_variation\load_variation_10.csv
 ./generated_data\output_11.csv complete, save to load_variation\load_variation_11.csv
 ./generated_data\output_12.csv complete, save to load_variat

In [2]:
import os
import pandas as pd


def split_and_save_data(X_train_file, Y_train_file, output_dir):
    
    X_train = pd.read_csv(X_train_file)
    Y_train = pd.read_csv(Y_train_file)

    
    if len(X_train) != len(Y_train):
        raise ValueError("error")

    
    os.makedirs(output_dir, exist_ok=True)

    
    for i in range(24):
        
        extracted_X = X_train[(X_train.index - i) % 24 == 0]
        extracted_Y = Y_train[(Y_train.index - i) % 24 == 0]

        
        if len(extracted_X) != len(extracted_Y):
            raise ValueError(f"hour{i}error")

        
        X_output_file = os.path.join(output_dir, f'X_train_hour_{i + 1}.csv')
        Y_output_file = os.path.join(output_dir, f'Y_train_hour_{i + 1}.csv')
        extracted_X.to_csv(X_output_file, index=False)
        extracted_Y.to_csv(Y_output_file, index=False)

X_train_file = './X_train.csv'  
Y_train_file = './Y_train.csv'  
output_dir = './hourly_data'    


split_and_save_data(X_train_file, Y_train_file, output_dir)

In [3]:
import pandas as pd
import os

def split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7):
    
    X_test = pd.read_csv(X_test_file)
    Y_test = pd.read_csv(Y_test_file)

    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    
    for hour in range(24):
        
        actual_hour = (start_hour - 1 + hour) % 24 + 1

        
        extracted_X = X_test.iloc[hour::24, :]
        extracted_Y = Y_test.iloc[hour::24, :]

        
        extracted_X.to_csv(f'{output_folder}/X_test_hour_{actual_hour}.csv', index=False)
        extracted_Y.to_csv(f'{output_folder}/Y_test_hour_{actual_hour}.csv', index=False)



def main():
    X_test_file = 'X_test.csv'
    Y_test_file = 'Y_test.csv'

    output_folder = './test_split'

    split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7)

main()

In [4]:
import os
import pandas as pd


base_dir = './hour'


for folder_num in range(1, 25):
    folder_path = os.path.join(base_dir, str(folder_num))
    
    cost_diff_path = os.path.join(folder_path, 'cost_differences.csv')
    cost_diff_df = pd.read_csv(cost_diff_path)
    
    perfect_cost_path = os.path.join(folder_path, 'perfect_costs.csv')
    perfect_cost_df = pd.read_csv(perfect_cost_path)
    
    result_df = cost_diff_df.div(perfect_cost_df['Perfect_Cost'], axis=0)
    
    result_path = os.path.join(folder_path, 'divided_costs.csv')
    result_df.to_csv(result_path, index=False)
    
    print(f"complete: {folder_path}")



complete: ./hour\1
complete: ./hour\2
complete: ./hour\3
complete: ./hour\4
complete: ./hour\5
complete: ./hour\6
complete: ./hour\7
complete: ./hour\8
complete: ./hour\9
complete: ./hour\10
complete: ./hour\11
complete: ./hour\12
complete: ./hour\13
complete: ./hour\14
complete: ./hour\15
complete: ./hour\16
complete: ./hour\17
complete: ./hour\18
complete: ./hour\19
complete: ./hour\20
complete: ./hour\21
complete: ./hour\22
complete: ./hour\23
complete: ./hour\24


In [5]:
import os
import pandas as pd


load_variation_dir = './load_variation'
hourly_data_dir = './hourly_data'


for i in range(1, 25):
    
    load_variation_file = os.path.join(load_variation_dir, f'load_variation_{i}.csv')
    y_train_file = os.path.join(hourly_data_dir, f'Y_train_hour_{i}.csv')
    
    
    load_variation_df = pd.read_csv(load_variation_file)
    
    
    y_train_df = pd.read_csv(y_train_file)
    y_train_column = y_train_df.iloc[:, 0]  
    
    
    result_df = load_variation_df.div(y_train_column, axis=0)
    
    
    result_file = os.path.join(load_variation_dir, f'load_variation_divided_{i}.csv')
    result_df.to_csv(result_file, index=False)
    